In [1]:
import pandas as pd
from urllib.request import urlretrieve
import os

In [2]:
urls = ['https://www.abs.gov.au/statistics/people/population/regional-population-age-and-sex/2021/32350DS0001_2021.xlsx', \
       'https://www.abs.gov.au/statistics/people/population/deaths-australia/2020/33020DO005_2020.xlsx', \
       'https://www.abs.gov.au/statistics/people/education/schools/2021/Table%2035b%20Counts%20of%20all%20Schools%2C%202010-2021.xlsx']

names = ['age_sex', 'pop_deaths', 'education']

In [3]:
# Creates raw_data folder
output_relative_dir = '../raw_data/'
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)
    
# Creates abs_data folder within raw_data
if not os.path.exists(output_relative_dir + 'abs_data'):
    os.makedirs(output_relative_dir + 'abs_data')

In [4]:
abs_output_dir = output_relative_dir + 'abs_data'

In [5]:
i = 0
for url in urls:
    output_dir = f"{abs_output_dir}/{names[i]}.xlsx"
    urlretrieve(url, output_dir)
    i += 1

# Manual steps required:

- Open the age_sex and pop_deaths .xlsx files
- 

## Reading in pop_deaths -

In [8]:
sheets = [2]
''', 2, 3, 4, 5, 6, 7'''
df = pd.read_excel('../raw_data/abs_data/pop_deaths.xlsx', sheet_name = sheets)

In [119]:
df = pd.read_excel('../raw_data/abs_data/pop_deaths.xlsx', sheet_name=1, skiprows=4, header=[0,1], nrows=129)[1:]
df = df.drop([(y, 'Standardised death rate.1') for y in range(2012, 2020)], axis=1)
df = df.drop([(y, 'Standardised death rate') for y in range(2012, 2021)], axis=1)
df.columns = [('LGA Code'), ('LGA Name')] + [','.join([str(c) for c in x]) for x in df.columns[2:]]
df2 = df.melt(id_vars=["LGA Code", "LGA Name"])
df2['value'] = df2.value.astype('int')
df2[['year','var']] = df2['variable'].str.split(',').tolist()
df2 = df2.sort_values('year')

In [129]:
# import seaborn as sns
# sns.scatterplot(df2.query("var=='Estimated resident population'"), x='year', y='value', hue='LGA Name')

In [120]:
df2.query("var=='Estimated resident population'").groupby(['LGA Code', 'LGA Name'])['value'].apply(pd.Series.pct_change)

0            NaN
126          NaN
34           NaN
35           NaN
36           NaN
          ...   
2137   -0.007213
2138    0.012455
2139    0.015675
2140    0.017003
2174    0.001449
Name: value, Length: 1152, dtype: float64